### CG Symmetric GS solver subroutine

In [4]:
import numpy as np
import math
from scipy.sparse import diags
from itertools import combinations

In [5]:
def generate_subroutine_bulk():
    ''' make 27pt stencil - bulk 3d '''
    outstring = "//bulk\n"
    outstring += "else{\n" 
    outstring += "xv[ix+iy*nx+iz*ny*nx] = rv[ix+iy*nx+iz*ny*nx]\n"
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                if i==j and j==k and i==0:
                    continue
                else:
                    outstring+="+ xv[ix+{} + (iy+{})*nx + (iz+{})*nx*ny]\n".format(i,j,k)
    outstring += ";}\n"
    return outstring

In [6]:
def zero_index_27pt():
    ''' corners '''
    def make_tempstring(l1,u1,l2,u2,l3,u3):
        tempstring = "xv[ix+iy*nx+iz*ny*nx] = rv[ix+iy*nx+iz*ny*nx] \n"
        for n0 in range(l1,u1):
            for n1 in range(l2,u2):
                for n2 in range(l3,u3):
                    if(n0 == 0 and n1 ==n0 and n2==n1):
                        continue
                    tempstring += "+xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
                    .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
        tempstring += ";}\n"
        return tempstring
    i1,i2,i3 = 'x', 'y', 'z'
    outstring = ""

    def set_value(val, indx):
        if val == 0:
            return "i{} == 0".format(indx,indx)
        elif val == 1:
            return "i{} == n{}-1".format(indx,indx,indx)

    for ia in range(2):
        for ib in range(2):
            for ic in range(2):
                if (ia == 0 and ib == 0 and ic ==0):
                    outstring += "if("
                else:
                    outstring += "else if("
                outstring += set_value(ia, 'x') + "&& "
                outstring += set_value(ib, 'y') + "&& "
                outstring += set_value(ic, 'z') + "){\n"
                outstring += make_tempstring(-ia,-ia+2,-ib,-ib+2,-ic,-ic+2)

    return outstring

In [7]:
def one_index_27pt():
    ''' edges '''
    outstring = "//edges \n"
    dims = 'xyz'
    for c in combinations(dims, 2):
        i1,i2,i3 = [i for i in dims if i not in c][0],c[0], c[1]
        def make_tempstring(l1,u1,l2,u2):
            tempstring = "i{}>0 && i{}<n{}-1)".format(i1,i1,i1,i1) + "{\n"
            tempstring += "xv[ix+iy*nx+iz*ny*nx] = rv[ix+iy*nx+iz*ny*nx] \n"
            for n0 in range(-1,2):
                for n1 in range(l1,u1):
                    for n2 in range(l2,u2):
                        if(n0 == 0 and n1 ==n0 and n2==n1):
                            continue
                        tempstring += "+xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
                        .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
            tempstring += ";}\n"
            return tempstring
        outstring += "else if (i{} == 0 && ".format(i2)
        outstring += "i{} == 0 &&".format(i3)
        outstring += make_tempstring(0,2,0,2)
        outstring += "else if (i{} == 0 && ".format(i2)
        outstring += "i{} == n{}-1 &&".format(i3,i3)
        outstring += make_tempstring(0,2,-1,1)
        outstring += "else if (i{} == n{}-1 && ".format(i2,i2)
        outstring += "i{} == 0 && ".format(i3)
        outstring += make_tempstring(-1,1,0,2)
        outstring += "else if (i{} == n{}-1 && ".format(i2,i2)
        outstring += "i{} == n{}-1 &&".format(i3,i3)
        outstring += make_tempstring(-1,1,-1,1)
        
    return outstring

In [8]:
def two_index_27pt():
    ''' sides '''
    outstring = "//sides \n"
    dims = 'xyz'
    for c in combinations(dims, 2):
        i1,i2,i3 = [i for i in dims if i not in c][0],c[0], c[1]
        def make_tempstring(l2,u2):
            tempstring = "i{}>0 && i{}<n{}-1 && ".format(i2,i2,i2,i2)
            tempstring += "i{}>0 && i{}<n{}-1)".format(i3,i3,i3,i3) + "{\n"
            tempstring += "xv[ix+iy*nx+iz*ny*nx] = rv[ix+iy*nx+iz*ny*nx] \n"
            for n0 in range(-1,2):
                for n1 in range(-1,2):
                    for n2 in range(l2,u2):
                        if(n0 == 0 and n1 ==n0 and n2==n1):
                            continue
                        tempstring += "+xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i2, "({}+{})".format("i"+i2,n0))\
                        .replace("i"+i3, "({}+{})".format("i"+i3,n1)).replace("i"+i1, "({}+{})".format("i"+i1,n2))
            tempstring += ";}\n"
            return tempstring
        outstring += "else if (i{} == 0 && ".format(i1)
        outstring += make_tempstring(0,2)
        outstring += "else if (i{} == n{}-1 && ".format(i1,i1)
        outstring += make_tempstring(-1,1)
        
    return outstring

In [42]:
def start_blurb_forward():
    outstring = '''//@HEADER
// ***************************************************
//
// HPCG: High Performance Conjugate Gradient Benchmark
//
// Contact:
// Michael A. Heroux ( maherou@sandia.gov)
// Jack Dongarra     (dongarra@eecs.utk.edu)
// Piotr Luszczek    (luszczek@eecs.utk.edu)
//
// ***************************************************
//@HEADER

/*!
 @file ComputeSYMGS_ref.cpp
 HPCG routine
 */

#ifndef HPCG_NO_MPI
#include "ExchangeHalo.hpp"
#endif
#include "ComputeSYMGS_ref.hpp"
#include <cassert>

/*!
  Computes one step of symmetric Gauss-Seidel:
  Assumption about the structure of matrix A:
  - Each row 'i' of the matrix has nonzero diagonal value whose address is matrixDiagonal[i]
  - Entries in row 'i' are ordered such that:
       - lower triangular terms are stored before the diagonal element.
       - upper triangular terms are stored after the diagonal element.
       - No other assumptions are made about entry ordering.
  Symmetric Gauss-Seidel notes:
  - We use the input vector x as the RHS and start with an initial guess for y of all zeros.
  - We perform one forward sweep.  x should be initially zero on the first GS sweep, but we do not attempt to exploit this fact.
  - We then perform one back sweep.
  - For simplicity we include the diagonal contribution in the for-j loop, then correct the sum after
  @param[in] A the known system matrix
  @param[in] r the input vector
  @param[inout] x On entry, x should contain relevant values, on exit x contains the result of one symmetric GS sweep with r as the RHS.
  @warning Early versions of this kernel (Version 1.1 and earlier) had the r and x arguments in reverse order, and out of sync with other kernels.
  @return returns 0 upon success and non-zero otherwise
  @see ComputeSYMGS
*/
int ComputeSYMGS_ref( const SparseMatrix & A, const Vector & r, Vector & x) {

  assert(x.localLength==A.localNumberOfColumns); // Make sure x contain space for halo values

#ifndef HPCG_NO_MPI
  ExchangeHalo(A,x);
#endif

  local_int_t ix = 0;
  local_int_t iy = 0;
  local_int_t iz = 0;
  local_int_t nex = 0;
  local_int_t ney = 0;
  local_int_t nez = 0;
  local_int_t nx = A.geom->nx;
  local_int_t ny = A.geom->ny;
  local_int_t nz = A.geom->nz;
  local_int_t nlocal = nx*ny*nz;
  int npx          = A.geom->npx;
  int npy          = A.geom->npy;
  int npz          = A.geom->npz;
  int ipx          = A.geom->ipx;
  int ipy          = A.geom->ipy;
  int ipz          = A.geom->ipz;
  double diagonal_element = A.matrixDiagonal[0][0];// big assumption?
//  const local_int_t nrow = A.localNumberOfRows;
//  double ** matrixDiagonal = A.matrixDiagonal;  // An array of pointers to the diagonal entries A.matrixValues
  const double * const rv = r.values;
  double * const xv = x.values;

// forward sweep
for( iz=0; iz< nz; iz++){
    for( iy=0; iy< ny; iy++){
        for (ix=0; ix < nx; ix++){
'''
    return outstring

In [10]:
def start_blurb_backwards():
    return '''// backwards sweep
for( iz=nz-1; iz>=0; iz--){
    for( iy=ny-1; iy>=0; iy--){
        for (ix=nx-1; ix>=0; ix--){
'''

In [48]:
def end_blurb():
    return '''      
xv[ix+iy*nx+iz*ny*nx] /= diagonal_element;
        }
      }
    }
'''
    return outstring

In [12]:
def generate_offset(x_min, x_max, y_min, y_max, z_min, z_max):
    '''R(right)L are x , FB are y, UD are z
       generate dictionary of dictionaries
         first is label to denote which movements are present (RLFBUD, U, etc)
         second relates particular movement, within that set, to an offset 
         eg offset_dict['RFU']['001'] would be the offset for the cell in +ve z 
           direction in RFU corner of
           MPI cell (x and y) also available in this decomposition of global grid '''
    offset_dict = {}
    total='0'
    for ic in range(z_min,z_max+1):
        for ib in range(y_min,y_max+1):        
            for ia in range(x_min,x_max+1):
                if (ia==0 and ib==0 and ic==0):continue
                offset_dict["{}{}{}".format(ia,ib,ic)]="{}".format(total)
                if(abs(ia)==1 and abs(ib)==1 and abs(ic)==1):
                    total += '+1'
                if(abs(ia)==1 and abs(ib)==1 and ic==0):
                    total += '+nz'
                if(abs(ia)==0 and abs(ib)==1 and abs(ic)==1):
                    total += '+nx'
                if(abs(ia)==1 and abs(ib)==0 and abs(ic)==1):
                    total += '+ny'
                if(abs(ia)==1 and abs(ib)==0 and abs(ic)==0):
                    total += '+ny*nz'
                if(abs(ia)==0 and abs(ib)==1 and abs(ic)==0):
                    total += '+nx*nz'
                if(abs(ia)==0 and abs(ib)==0 and abs(ic)==1):
                    total += '+nx*ny'                    
    return offset_dict

In [13]:
def general_indices_face(quick_axis, slow_axis, quick_posn, slow_posn):
    ''' alphabetically quick axis is lower '''
    assert(quick_axis < slow_axis)
    if (quick_posn == -1 and slow_posn == -1):
        return("", "+1", "+n{}".format(quick_axis), "+n{}+1".format(quick_axis))
    if (quick_posn == -1 and slow_posn == 1):
        return("+n{}*(n{}-2)".format  (quick_axis, slow_axis),
               "+n{}*(n{}-2)+1".format(quick_axis, slow_axis),
               "+n{}*(n{}-1)".format  (quick_axis, slow_axis),
               "+n{}*(n{}-1)+1".format(quick_axis, slow_axis))
    if (quick_posn == 1 and slow_posn == -1):
        return("+n{}-2".format  (quick_axis),
               "+n{}-1".format  (quick_axis),
               "+2*n{}-2".format(quick_axis),
               "+2*n{}-1".format(quick_axis))
    if (quick_posn == 1 and slow_posn == 1):
        return("+n{}*(n{}-1)-2".format(quick_axis, slow_axis),
               "+n{}*(n{}-1)-1".format(quick_axis, slow_axis),
               "+n{}*n{}-2".format    (quick_axis, slow_axis),
               "+n{}*n{}-1".format    (quick_axis, slow_axis))

In [14]:
def general_indices_edge(axis, posn):
    ''' axis is string name, posn is -1 for start 1 for end '''
    if posn==-1:
        return("", "+1")
    elif posn==1:
        return("+n{}-2".format(axis), "+n{}-1".format(axis))
    else:
        assert(posn in [-1,1])

In [15]:
def corner_to_bulk(a,b,c, lower_a, upper_a, lower_b, upper_b, lower_c, upper_c, extra_offset):
    ''' a,b,c define corner - search further in this direction if bounds allow
     rename to corner_extend if a general sub can be written '''
    outstring = ""
    for ia in range(lower_a,upper_a + 1):
        for ib in range(lower_b,upper_b + 1):
            for ic in range(lower_c,upper_c + 1):
                #can't extend corner in opposite direction
                if ((ia == 0) and (ib==0) and (ic==0)):
                    continue
                #need at least 1 corner to match extension direction to add these
                if not ((a == ia) or (b == ib) or (c==ic)):
                    continue
#                if (abs(ia)==1 and abs(ib)==1 and abs(ic)==1):
#                    outstring += "-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]+"]\n"
                # 1 zero (faces)
                if(abs(ia)==1 and ib==0 and ic==0):
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_face('y','z',b,c)])
                if(ia==0 and abs(ib)==1 and ic==0):
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_face('x','z',a,c)])
                if(ia==0 and ib==0 and abs(ic)==1):
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_face('x','y',a,b)])

                #need 2 corners to match extension direction to add these
                if not (((a == ia)and(b == ib)) or ((a == ia)and(c==ic))\
                      or((b == ib)and(c == ic))):
                    continue                      
                if (abs(ia) ==0 and abs(ib)==1 and abs(ic)==1):# add 2
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_edge('x',a)])
                if (abs(ia) ==1 and abs(ib)==0 and abs(ic)==1):# add 2
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_edge('y',b)])
                if (abs(ia) ==1 and abs(ib)==1 and abs(ic)==0):# add 2
                     outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]\
                      +"{}]\n".format(i) for i in general_indices_edge('z',c)])
                #need 3 corners to match extension direction to add these
                if ((a == ia)and(b == ib)and(c==ic)):
                     outstring+="-xv[nlocal+"+extra_offset[str(ia)+str(ib)+str(ic)]+"]\n"
    outstring+=";\n"
    return outstring

In [60]:
def edge_to_bulk(a, lower_b, upper_b, lower_c, upper_c, extra_offset):
    ''' a, defines edge (x,y,z) - search further other directions if bounds allow '''
    other_two_axes = {'x':('y','z'),'y':('x','z'),'z':('x','y')}

    outstring = ""

    for ib in range(lower_b,upper_b + 1): #look for ways to expand in quicker direction
        for ic in range(lower_c,upper_c + 1):
            if(abs(ib)==1 and abs(ic)==0): #moving in quicker direction of b and c

                if a == 'x':
                    outstring+="else if("
                    if(ib==-1): #y backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="&&i{}==0)\n".format(other_two_axes[a][1])#z=0
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','z',-1)])
                    outstring+=";}"
                    outstring+="else if("
                    if(ib==-1): #y backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="&&i{}==n{}-1)\n".format(other_two_axes[a][1],other_two_axes[a][1])#z=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','z',1)])
                    outstring+=";}\n"

                if a == 'y':
                    outstring+="else if("
                    if(ib==-1): #x backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #x forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="&&i{}==0)\n".format(other_two_axes[a][1])
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','y','z',-1)])
                    outstring+=";}"
                    outstring+="else if("
                    if(ib==-1): #x backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #x forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="&&i{}==n{}-1)\n".format(other_two_axes[a][1],other_two_axes[a][1])#z=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','y','z',1)])
                    outstring+=";}\n"

                if a == 'z':
                    outstring+="else if("
                    if(ib==-1): #y backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="&&i{}==0)\n".format(other_two_axes[a][1])#z=0
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','y','z',-1)])
                    outstring+=";}"
                    outstring+="else if("
                    if(ib==-1): #y backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    outstring+="&&i{}==n{}-1)\n".format(other_two_axes[a][1],other_two_axes[a][1])#z=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','y','z',1)])
                    outstring+=";}\n"

            if(abs(ib)==0 and abs(ic)==1): #moving in slower direction of b and c
                if a == 'x':
                    outstring+="else if("                   
                    if(ic==-1): #z backwards
                        outstring+="i{}==0".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+="&& i{}==0)".format(other_two_axes[a][0])#y=0
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','y',-1)])
                    outstring+=";}"
                    outstring+="else if("                   
                    if(ic==-1): #z backwards
                        outstring+="i{}==0".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+="&& i{}==n{}-1)".format(other_two_axes[a][0],other_two_axes[a][0])#y=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('x','x','y',1)])
                    outstring+=";}\n"

                if a == 'y':
                    outstring+="else if("                   
                    if(ic==-1): #z backwards
                        outstring+="i{}==0".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+=" && i{}==0)".format(other_two_axes[a][0])#y=0
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','x','y',-1)])
                    outstring+=";}"
                    outstring+="else if("                   
                    if(ic==-1): #z backwards
                        outstring+="i{}==0".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1           
                    outstring+="&&i{}==n{}-1)".format(other_two_axes[a][0],other_two_axes[a][0])#y=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('y','x','y',1)])
                    outstring+=";}\n"

                if a == 'z':
                    outstring+="else if("                   
                    if(ic==-1): #z backwards
                        outstring+="i{}==0".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1
                    outstring+="&&i{}==0)".format(other_two_axes[a][0])#y=0
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','x','z',-1)])
                    outstring+=";}"
                    outstring+="else if("                   
                    if(ic==-1): #z backwards
                        outstring+="i{}==0".format(other_two_axes[a][1])#set y=0
                    if(ic==1): #z forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][1],other_two_axes[a][1])#set y=ny-1      
                    outstring+="&&i{}==n{}-1)".format(other_two_axes[a][0],other_two_axes[a][0])#y=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                          +"{}]\n".format(i) for i in expand_edge_to_face('z','x','z',1)])
                    outstring+=";}\n"
                  

            if(abs(ib)==1 and abs(ic)==1):
                if a == 'x':
                    outstring+="else if("
                    if(ib==-1): #y backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    if(ic==-1): #z backwards
                        outstring+="&&i{}==0)".format(other_two_axes[a][1])#set z=0
                    if(ic==1): #z forwards
                        outstring+="&&i{}==n{}-1)".format(other_two_axes[a][1],other_two_axes[a][1])#set z=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ib)+str(ic)]\
                             +"{}]\n".format(i) for i in expand_edge_to_edge(a)])
                    outstring+=";}\n"
                if a == 'y':
                    outstring+="else if("
                    if(ib==-1): 
                        outstring+="i{}==0".format(other_two_axes[a][0])
                    if(ib==1): 
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    if(ic==-1):
                        outstring+="&&i{}==0)".format(other_two_axes[a][1])
                    if(ic==1): 
                        outstring+="&&i{}==n{}-1)".format(other_two_axes[a][1],other_two_axes[a][1])#set z=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(0)+str(ic)]\
                             +"{}]\n".format(i) for i in expand_edge_to_edge(a)])
                    outstring+=";}\n"
                if a == 'z':
                    outstring+="else if("
                    if(ib==-1): #y backwards
                        outstring+="i{}==0".format(other_two_axes[a][0])#set y=0
                    if(ib==1): #y forwards
                        outstring+="i{}==n{}-1".format(other_two_axes[a][0],other_two_axes[a][0])#set y=ny-1
                    if(ic==-1): #z backwards
                        outstring+="&&i{}==0)".format(other_two_axes[a][1])#set z=0
                    if(ic==1): #z forwards
                        outstring+="&&i{}==n{}-1)".format(other_two_axes[a][1],other_two_axes[a][1])#set z=nz-1
                    #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(a,a,a,a)
                    outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
                    outstring+="".join(["-xv[nlocal+"+extra_offset[str(ib)+str(ic)+str(0)]\
                             +"{}]\n".format(i) for i in expand_edge_to_edge(a)])
                    outstring+=";}\n"
                    
    #outstring += ";\n"                
    return outstring

In [17]:
def expand_edge_to_face(edge_axis, quick_axis, slow_axis, edge_posn):
    ''' alphabetically quick axis is lower 
        note face is made of edge and other ()'''
    assert(quick_axis < slow_axis)
#    outstring = "for( i{}=1; i{}< n{}-1; i{}++){\n".format(edge_axis, edge_axis, edge_axis) 
    #axis is i_edge*quick axis
    if (edge_axis == quick_axis and edge_posn == -1):
        return("+i{}-1"     .format(edge_axis), 
               "+i{}"       .format(edge_axis),
               "+i{}+1"     .format(edge_axis),
               "+n{}+i{}-1".format(quick_axis, edge_axis),
               "+n{}+i{}"  .format(quick_axis, edge_axis),
               "+n{}+i{}+1".format(quick_axis, edge_axis))
    if (edge_axis == quick_axis and edge_posn == 1):
#    if (edge_axis == quick_axis and slow_posn == 1):
        return("+i{}+(n{}-2)*n{}-1".format(edge_axis, slow_axis, quick_axis), 
               "+i{}+(n{}-2)*n{}"  .format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-2)*n{}+1".format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-1)*n{}-1".format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-1)*n{}"  .format(edge_axis, slow_axis, quick_axis),
               "+i{}+(n{}-1)*n{}+1".format(edge_axis, slow_axis, quick_axis))
#    if (edge_axis == slow_axis and quick_posn == -1):
    if (edge_axis == slow_axis and edge_posn == -1):
        return("+(i{}-1)*n{}"  .format(edge_axis, quick_axis),
               "+(i{}-1)*n{}+1".format(edge_axis, quick_axis),
               "+(i{})*n{}"    .format(edge_axis, quick_axis),
               "+(i{})*n{}+1".format(edge_axis, quick_axis),
               "+(i{}+1)*n{}"  .format(edge_axis, quick_axis),
               "+(i{}+1)*n{}+1".format(edge_axis, quick_axis))
#    if (edge_axis == slow_axis and quick_posn == 1):
    if (edge_axis == slow_axis and edge_posn == 1):
        return("+(i{}-1)*n{}+n{}-2".format(edge_axis, quick_axis, quick_axis),
               "+(i{}-1)*n{}+n{}-1".format(edge_axis, quick_axis, quick_axis),
               "+(i{})*n{}+n{}-2".format(edge_axis, quick_axis, quick_axis),
               "+(i{})*n{}+n{}-1".format(edge_axis, quick_axis, quick_axis),
               "+(i{}+1)*n{}+n{}-2".format(edge_axis, quick_axis, quick_axis),
               "+(i{}+1)*n{}+n{}-1".format(edge_axis, quick_axis, quick_axis))

In [18]:
def expand_edge_to_edge(edge_axis):
    ''' Add 3 more points from edge '''
    return("+i{}-1".format(edge_axis),"+i{}".format(edge_axis),"+i{}+1".format(edge_axis))

In [35]:
def face_to_bulk(a, lower_a, upper_a, extra_offset):
    other_two_axes = {'x':('y','z'),'y':('x','z'),'z':('x','y')}
    outstring = ""

    for ia in range(lower_a,upper_a + 1):
        if (ia == 0):
            continue
        if (ia == 1):
            outstring += "else if (i{}==n{}-1)".format(a,a)
        if (ia == -1):
            outstring += "else if (i{}==0)".format(a)
        #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(other_two_axes[a][0],
        #                                                   other_two_axes[a][0],
        #                                                   other_two_axes[a][0],
        #                                                   other_two_axes[a][0])
        #outstring+="{\n"
        #outstring+="for (i{}=1; i{}<n{}-1;i{}++)\n".format(other_two_axes[a][1],
        #                                                   other_two_axes[a][1],
        #                                                   other_two_axes[a][1],
        #                                                   other_two_axes[a][1])
        # add 9 points
        outstring+="{\n xv[ix+iy*nx+iz*ny*nx] +=\n"
        if (a=='x'):
            outstring+="".join(["-xv[nlocal+"+extra_offset[str(ia)+str(0)+str(0)]\
               +"{}]\n".format(i) for i in expand_face_to_neighbour(*other_two_axes[a])])
        if (a=='y'):
            outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(ia)+str(0)]\
               +"{}]\n".format(i) for i in expand_face_to_neighbour(*other_two_axes[a])])
        if (a=='z'):
            outstring+="".join(["-xv[nlocal+"+extra_offset[str(0)+str(0)+str(ia)]\
               +"{}]\n".format(i) for i in expand_face_to_neighbour(*other_two_axes[a])])
        outstring+=";}\n"

    return outstring

In [20]:
def expand_face_to_neighbour(quick_axis, slow_axis):
    ''' '''
    return("+i{}-1+(i{}-1)*n{}".format(quick_axis, slow_axis, quick_axis),
           "+i{}+(i{}-1)*n{}"  .format(quick_axis, slow_axis, quick_axis),
           "+i{}+1+(i{}-1)*n{}".format(quick_axis, slow_axis, quick_axis),
           "+i{}-1+(i{})*n{}"  .format(quick_axis, slow_axis, quick_axis),
           "+i{}+i{}*n{}"      .format(quick_axis, slow_axis, quick_axis),
           "+i{}+1+i{}*n{}"    .format(quick_axis, slow_axis, quick_axis),
           "+i{}-1+(i{}+1)*n{}".format(quick_axis, slow_axis, quick_axis),
           "+i{}+(i{}+1)*n{}"  .format(quick_axis, slow_axis, quick_axis),
           "+i{}+1+(i{}+1)*n{}".format(quick_axis, slow_axis, quick_axis))

In [46]:
def zero_index_27pt_mpi(lower_x, upper_x, lower_y, upper_y, lower_z, upper_z, offset_dict):
    ''' corners '''
    i1,i2,i3 = 'x', 'y', 'z'
    outstring = ""

    def set_value(val, indx):
        if val == 0:
            return "i{} == 0".format(indx)
        elif val == 1:
            return "i{} == n{}-1".format(indx,indx)

    # first time use if, all other times use else if
    if_string = "if("
    for ia in range(2):
        for ib in range(2):
            for ic in range(2):
                #if (ia, ib, )
                added_terms = corner_to_bulk(ia*2-1, ib*2-1, ic*2-1,
                             lower_x, upper_x, lower_y,upper_y, lower_z, upper_z, offset_dict)
                #
                if added_terms == ";\n":
                    continue
#                if (ia == 0 and ib == 0 and ic ==0):
#                    outstring += "if("
#                else:
#                    outstring += "else if("
                outstring += if_string
                if_string = "else if("
                outstring += set_value(ia, 'x') + "&& "
                outstring += set_value(ib, 'y') + "&& "
                outstring += set_value(ic, 'z') + "){\n"
                outstring += "xv[ix+iy*nx+iz*ny*nx] += \n"
                outstring += added_terms + "}\n"
    return outstring

In [22]:
def one_index_27pt_mpi(lower_x, upper_x, lower_y, upper_y, lower_z, upper_z, offset_dict):
    ''' edges - assume all mpi ranks'''

    outstring = ""
    for a in [('x', lower_y, upper_y, lower_z, upper_z, offset_dict),
              ('y', lower_x, upper_x, lower_z, upper_z, offset_dict), 
              ('z', lower_x, upper_x, lower_y, upper_y, offset_dict) ]:

        outstring += edge_to_bulk(*a)

    return outstring

In [23]:
def two_index_27pt_mpi(lower_x, upper_x, lower_y, upper_y, lower_z, upper_z, offset_dict):
    ''' faces - include more mpi ranks'''

    outstring = ""
    for a in [('x', lower_x, upper_x, offset_dict),
              ('y', lower_y, upper_y, offset_dict), 
              ('z', lower_z, upper_z, offset_dict) ]:

        outstring += face_to_bulk(*a)

    return outstring

In [54]:
def print_extra_MPI_contributions():
    outstring=""
    for xl in [['if(ipx > 0)',-1], ['else',0]]:
        outstring+=xl[0] + "\n"
        outstring+='{\n'
        for xh in [['if(ipx < npx - 1)',1], ['else',0]]:
            outstring+="  " + xh[0] + "\n"
            outstring+='{\n'
            for yl in [['if(ipy > 0)',-1], ['else',0]]:
                outstring+="    " + yl[0] + "\n"
                outstring+='{\n'
                for yh in [['if(ipy < npy - 1)',1], ['else',0]]:
                    outstring+="      " + yh[0] + "\n"
                    outstring+='{\n'
                    for zl in [['if(ipz > 0)',-1], ['else',0]]:
                        outstring+="        " + zl[0] + "\n"
                        outstring+='{\n'
                        for zh in [['if(ipz < npz - 1)',1], ['else',0]]:
                            outstring+="            "+ zh[0] + "\n"
                            outstring+='{\n'
                            bounds = (xl[1],xh[1],yl[1],yh[1],zl[1],zh[1])
                            offset_dict = generate_offset(*bounds)
                            outstring += zero_index_27pt_mpi(*bounds, offset_dict)
                            outstring += one_index_27pt_mpi (*bounds, offset_dict)
                            outstring += two_index_27pt_mpi (*bounds, offset_dict)

                            outstring+="}//ipz < npz - 1\n"
                        outstring+='          }//ipz > 0 \n'
                    outstring+='        }//ipy < npy - 1 \n'
                outstring+='      }//ipy > 0 \n'
            outstring+='    }//ipx < npx - 1 \n'
        outstring+=' }//ipx > 0 \n'
    return outstring

In [62]:
def generate_subroutine():
    return start_blurb_forward() + "\n" \
         + zero_index_27pt() + "\n" \
         + one_index_27pt() + "\n" \
         + two_index_27pt() + "\n" \
         + generate_subroutine_bulk() + "\n" \
         + print_extra_MPI_contributions() + "\n" \
         + end_blurb() +"\n"\
         + start_blurb_backwards() + "\n" \
         + zero_index_27pt() + "\n" \
         + one_index_27pt() + "\n" \
         + two_index_27pt() + "\n" \
         + generate_subroutine_bulk() + "\n" \
         + end_blurb() +"\n"\
         + '''  
return 0;
}'''

In [63]:
with open('symgs_subroutine.cpp', 'w') as outfile:
    outfile.write(generate_subroutine())
#print(generate_subtroutine())

In [49]:
#print(generate_subtroutine())
#print(print_extra_MPI_contributions())